# Tensorflow学习
Tensorflow是谷歌推出的用于人工智能的开源神器，也是现在最流行的深度学习框架。但是由于框架在不断更新，整个项目也非常大，导致整体学下来不是那么轻松，我鼓起勇气学了三次，有两次都中途放弃了，后来实在要用到才坚持学了下去，由此可见，deadline才是第一生产力啊。

## 1、基本概念
Tensoflow是一个基于图模型的计算框架，这也就意味着在计算之前我们需要知道计算过程中需要进行哪些操作，然后将这些步骤在图中进行描述，最后对整个图进行数据的操作。最开始大家可能对构建图和实际计算的分离不太适应，但是这样的方法也使得整体计算思路比较清晰，而且能够将数据独立出来，形成一个通用的计算流程，也就是说，不论是图像数据还是正常数据，同一个图同一个算法都能够进行处理。   
除此之外，Tensorflow也能够自动的计算微分，从而进行梯度更新。这对深度学习算法非常的有利。

## 2、数据类型
由于Tensorflow设计理念的不同，它在数据类型方面存在着特殊之处，同时为了兼容，Tensorflow中许多生成数据的函数也都是从Numpy继承而来。Tensorflow中的数据类型可以分为三类（这里不是指float这种）：常量、变量、占位符

### 常量
常量是指在计算过程中不需要改变或者是为了方面而生成的一些数据。这里的许多函数与Numpy类似，ones,zeros,linspace,range,random_uniform等。

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
a = tf.constant([[1,2],[3,4]],dtype='float32')
b = tf.ones((2,2))
print("a:",a)
print("b:",b)
# 不输出数据结果，只输出节点类型，这也是我们上面说的图和计算分开

a: Tensor("Const:0", shape=(2, 2), dtype=float32)
b: Tensor("ones:0", shape=(2, 2), dtype=float32)


In [3]:
c = tf.matmul(a,b)
print('c:',c)

c: Tensor("MatMul:0", shape=(2, 2), dtype=float32)


In [4]:
# 为了计算实际的过程，我们需要进行图的构建和运行
sess = tf.Session()
result_c = sess.run(c)
print("result_c:\n",result_c)

result_c:
 [[3. 3.]
 [7. 7.]]


### 变量
变量是指在训练或者在执行过程中可以改变的量，一般来说可以用来表示神经网络权重值。变量的初始值可以指定，也可以随机给出。这里初始化分为两种：一种是变量Variable初始化需要指定shape，另一种是初始化器，会根据神经层自动确定shape

In [5]:
w = tf.Variable(tf.random_normal(shape=(1,3)))
x = tf.Variable(tf.ones(shape=(3,5)))
y = tf.matmul(w,x)
# 变量定义之后需要初始化
init = tf.global_variables_initializer()

In [6]:
# 构建图之后初始化并进行计算
sess = tf.Session()
sess.run(init)
sess.run(y)

array([[1.828585, 1.828585, 1.828585, 1.828585, 1.828585]], dtype=float32)

### 占位符
占位符是Tensorflow里比较特殊的一种类型，指的是我不管数据有多少个样本，但是数据的特征维度是固定的，这样在没有接受实际数据之前，我也能够将表示数据的图模型构建出来。这里使用的是tf.placeholder(dtype,shape=())，之后使用的时候需要告知这个填充什么数据，使用feed_dict来进行填充。

In [7]:
x = tf.placeholder(tf.float32, shape=(1024, 1024))
y = x**2
z = tf.matmul(x,x)
# 这是另一种构建图的方式，类似文件的打开
with tf.Session() as sess:
  rand_array = np.random.rand(1024, 1024)
  z_value,y_value = sess.run((z,y), feed_dict={x: rand_array})
print(z_value)
print(y_value)

[[250.90715 251.88919 258.58353 ... 248.58905 249.54204 251.23988]
 [258.9687  264.44757 267.3337  ... 257.73926 259.23145 259.9408 ]
 [248.12717 252.07909 255.57857 ... 243.87004 253.33029 249.88893]
 ...
 [247.75598 256.4641  254.49515 ... 252.31421 248.33751 252.99547]
 [253.04858 259.52606 265.12958 ... 254.40196 253.90582 259.59256]
 [253.64949 257.35117 268.2553  ... 254.38657 254.54013 262.23645]]
[[0.5149757  0.42899185 0.6539263  ... 0.85984874 0.15566608 0.1261262 ]
 [0.86850375 0.64628404 0.2555107  ... 0.59191906 0.03171698 0.49893692]
 [0.17778714 0.5731438  0.08192325 ... 0.87574875 0.11272709 0.12856431]
 ...
 [0.11933188 0.58362293 0.19241887 ... 0.01995767 0.11896862 0.4169119 ]
 [0.36336985 0.65014195 0.40851656 ... 0.1521543  0.71825266 0.7413794 ]
 [0.46060216 0.75984365 0.5192061  ... 0.17020163 0.09187417 0.30956146]]


In [8]:
np.random.rand(2)

array([0.50161312, 0.44462827])

## 3、神经网络的构建
神经网络在Tensorflow的构建算是比较方便的，特别是新版本的tensorflow增加了layer这样的封装。构建网络之前我们首先要明确几个概念：  
* 输入向量：输入向量有X和Y_True，由于这两种都是样本数量不定，所以都需要用placeholder来定义  
* 神经网络层：神经单元数、神经网络层数、激活函数  
* 损失函数：训练的损失函数的定义，是优化的指标  
* 训练：训练过程及优化方法

In [9]:
# 数据集准备
from sklearn import datasets
X = datasets.load_digits()['data']
Y = datasets.load_digits()['target']
print(X.shape,Y.shape)
from sklearn import preprocessing
# one-hot编码
Y = preprocessing.LabelBinarizer().fit_transform(Y)
Y = Y.astype('float64')
print(X.shape,Y.shape)

(1797, 64) (1797,)
(1797, 64) (1797, 10)


In [10]:
# 将数据分为训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [11]:
# 定义图
x = tf.placeholder(dtype='float64', shape=[None, 64], name='x')
y = tf.placeholder(dtype='float64', shape=[None, 10], name='y')
l1 = tf.layers.dense(x,50,activation=tf.nn.relu)
l2 = tf.layers.dense(l1,50,activation=tf.nn.relu)
l3 = tf.layers.dense(l2,50,activation=tf.nn.relu)
y_pre = tf.layers.dense(l3,10,activation=tf.nn.softmax)
y_eva = tf.argmax(y,1)# 沿着纵轴
y_pre_eva = tf.argmax(y_pre,1)

In [12]:
# 定义损失函数和训练参数
loss = tf.reduce_mean(tf.square(y - y_pre))
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
init = tf.global_variables_initializer()

In [13]:
sess = tf.Session() 
sess.run(init)
for i in range(10000):
    _,loss_value = sess.run((train,loss),feed_dict={x:X_train,y:Y_train})
    if i% 500 == 0:    
        print(loss_value)

0.13900950929846173
0.07080129037480684
0.056790036926201964
0.05117441537207622
0.03991216593480311
0.03581079899805159
0.03398151939658355
0.032694728805448624
0.031688957274901876
0.030892379523433856
0.030225337165344437
0.029647886556648108
0.029149215643724365
0.028717186769375437
0.028341708662105607
0.028007121397433794
0.027710843458555342
0.02743916412051914
0.027153041227075885
0.019492803306579036


In [14]:
# 判断准确率
Y1 = sess.run(y_pre_eva,feed_dict={x:X_test})
Y2 = sess.run(y_eva,feed_dict={y:Y_test})
from sklearn import metrics
metrics.accuracy_score(Y1,Y2)

0.8703703703703703

这篇文章只是很简要的的介绍了Tensorflow的用法，我相信与知识相比更重要的是兴趣，因此这篇文章更多的是为了向跟我一样的初学者们展示Tensorflow的基本框架，仅作抛砖引玉。